# Lab 03: Spatial image processing

The purpose of this lab is to practice preprocessing needed on T1 images images and spatial normalization. 

## Preparation
* import python tools for plotting 3D images
* copy basic T1 and bold image from our osfshare for easy access to the files for this lab

In [1]:
from nilearn import plotting
from nilearn import image

In [2]:
%%bash
# Copy files from lab 01 for easier work with them here
# Name them generic names for ease of seeing how the command line programs work
cp ../01-Lab/osfshare/sub-97-T1w_brain.nii.gz my_T1_brain.nii.gz
cp ../01-Lab/osfshare/sub-97_task-flanker_bold.nii.gz my_bold.nii.gz

### How did we get that "brain" image with the skull stripped  again?
* FSL provides a tool called _bet_ which stands for _brain extraction tool_. See the [bet user manual here](https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/BET/UserGuide). 
* It requires only the brain image with skull as input and has relatively few options. Performance is generally OK, but it is common to need to tweak the -f and -g options. 
* If you want to get serious about getting a great brain mask (e.g., your brains have many individual differences in shape due to atrophy), you should look to [ANTs](http://stnava.github.io/ANTs/). Here is an [example](https://dpaniukov.github.io/2016/06/06/brain-extraction-with-ants.html) of using their brain extraction tool
* *Good brain extraction is essential to good registration*. If you see poor registration of your images later on then one place to start trouble-shooting is to look at the quality of your brain extraction by comparing the brain mask (what the program thinks is brain) to the image with the skull (did the program include non-brain tissues in the mask OR chop off part of the brain?)
    - Never assume BET worked great, always check!!

In [3]:
# refresher, are my images here?
!ls

my_bold.nii.gz	my_T1_brain.nii.gz  SpatialImageProcessing.ipynb


## T1 image bias correction and image segmentation
* FSL provides a nice tool called [FAST](https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/FAST) that combines bias field correction (see Handbook p56) and tissue segmentation
* You will need to input a T1 image of the brain without skull, so brain extraction should be run before-hand

In [5]:
# Look at usage
!fast


Part of FSL (build 509)
FAST 
Copyright(c) 2004-2012, University of Oxford

Usage: 
fast [options] file(s)

Optional arguments (You may optionally specify one or more of):
	-n,--class	number of tissue-type classes; default=3
	-I,--iter	number of main-loop iterations during bias-field removal; default=4
	-l,--lowpass	bias field smoothing extent (FWHM) in mm; default=20
	-t,--type	type of image 1=T1, 2=T2, 3=PD; default=T1
	-f,--fHard	initial segmentation spatial smoothness (during bias field estimation); default=0.02
	-g,--segments	outputs a separate binary image for each tissue type
	-a <standard2input.mat> initialise using priors; you must supply a FLIRT transform
	-A <prior1> <prior2> <prior3>    alternative prior images
	--nopve	turn off PVE (partial volume estimation)
	-b		output estimated bias field
	-B		output bias-corrected image
	-N,--nobias	do not remove bias field
	-S,--channels	number of input images (channels); default 1
	-o,--out	output basename
	-P,--Prior	use priors thr

In [7]:
%%bash
# example usage below
# running this should take ~10 minutes
fast -t 1 -n 3 -g -b -B -v -o my_T1_brain my_T1_brain.nii.gz > fast.log

In [ ]:
# Write out what these options are doing
# Verbose output can help with seeing under the hood and tracking the process

In [8]:
# Check outputs
!ls

fast.log		      my_T1_brain_pveseg.nii.gz
my_bold.nii.gz		      my_T1_brain_restore.nii.gz
my_T1_brain_bias.nii.gz       my_T1_brain_seg_0.nii.gz
my_T1_brain_mixeltype.nii.gz  my_T1_brain_seg_1.nii.gz
my_T1_brain.nii.gz	      my_T1_brain_seg_2.nii.gz
my_T1_brain_pve_0.nii.gz      my_T1_brain_seg.nii.gz
my_T1_brain_pve_1.nii.gz      SpatialImageProcessing.ipynb
my_T1_brain_pve_2.nii.gz


### Practice: 
* Use FSLview to open your T1 image with the "seg" images overlayed on top
* What do these images represent?

In [9]:
!fslview my_T1_brain.nii.gz my_T1_brain_seg_0.nii.gz my_T1_brain_seg_1.nii.gz my_T1_brain_seg_2.nii.gz

* The bias corrected image ends with "restore" - take a look and compare values in the view and by looking at their histograms

## Spatial registration
* Now let's walk through how to spatially align our high-resolution T1 image and our functional bold image to the MNI-152 standard template.

### Open MNI 152 template in FSLview
* Open standard > MNI152_T1_2mm_brain
* Where is origin?
* What does 2mm refer to in the file name?

In [11]:
%%bash
fslview